<a href="https://colab.research.google.com/github/seismosmsr/hawaii_soils/blob/main/Hawaii_Soils_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
install.packages('sf')
install.packages('raster')
install.packages('exactextractr')
install.packages("googledrive")
install.packages("googletoken")
install.packages("DBI")
install.packages("RSQLite")
install.packages("RPostgreSQL")
install.packages("terra")
install.packages('randomForest')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proxy’, ‘e1071’, ‘wk’, ‘classInt’, ‘Rcpp’, ‘s2’, ‘units’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘sp’, ‘terra’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘googletoken’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the

In [2]:
library(ggplot2)
library(sf)
library(magrittr)
library(data.table)
library(raster)
library(exactextractr)
library(googledrive)
library(DBI)
library(terra)
library(randomForest)
library(parallel)

Linking to GEOS 3.10.2, GDAL 3.4.3, PROJ 8.2.1; sf_use_s2() is TRUE

Loading required package: sp

terra 1.7.55


Attaching package: ‘terra’


The following object is masked from ‘package:data.table’:

    shift


The following objects are masked from ‘package:magrittr’:

    extract, inset


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin




In [3]:
drive_auth(use_oob = TRUE, cache = FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=45f2192d29f194ee3d963e6d6af0398e&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQWZKb2hYbC1DdW9xeWhYZUNOMEVHTWRHYTdMLVJlUmh4bW5FYmF5dmRfYXp1U1ZIN2hfVEZZWEhDeERCeXlOMHU0YUlCZyIsInN0YXRlIjoiNDVmMjE5MmQyOWYxOTRlZTNkOTYzZTZkNmFmMDM5OGUifQ==


In [37]:
# https://drive.google.com/file/d/1ryTzWz0t9mgI5ynPvMzM21D135dLJvWv/view?usp=drive_link
# Updated: https://drive.google.com/file/d/1-dAAC42MM2BrShxXR_SUhavcawIHYy6B/view?usp=drive_link
drive_download(as_id('1-dAAC42MM2BrShxXR_SUhavcawIHYy6B'), path = paste0('/content/fia.csv'), overwrite = TRUE)
fia_dt <-  st_read('/content/fia.csv') %>% st_as_sf(coords = c("LON","LAT"), crs = 4326) %>% data.table
fia_dt$county_name <- fia_dt$County.Name

File downloaded:

• 2023_11_01FIA_soildepths.csv <id: 1-dAAC42MM2BrShxXR_SUhavcawIHYy6B>

Saved locally as:

• /content/fia.csv



Reading layer `fia' from data source `/content/fia.csv' using driver `CSV'


Warning message:
“no simple feature geometries present: returning a data.frame or tbl_df”


In [17]:
# https://drive.google.com/file/d/1j4B8qx1S0nHLSYrnRVRckwFTX9f2z1nb/view?usp=drive_link
# updated: https://drive.google.com/file/d/1LZQIykiIRmkIv-Ygy24rAN1ohmGOG1rY/view?usp=drive_link
drive_download(as_id('1LZQIykiIRmkIv-Ygy24rAN1ohmGOG1rY'), path = "/content/250_summary_grid_dt.gpkg", overwrite = TRUE)
grid_dt <- st_read("/content/250_summary_grid_dt.gpkg") %>% st_transform(4326) %>% data.table()

File downloaded:

• 250_summary_grid_dt.gpkg <id: 1LZQIykiIRmkIv-Ygy24rAN1ohmGOG1rY>

Saved locally as:

• /content/250_summary_grid_dt.gpkg



Reading layer `250_summary_grid_dt' from data source 
  `/content/250_summary_grid_dt.gpkg' using driver `GPKG'
Simple feature collection with 227561 features and 38 fields
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: 57161.36 ymin: 16952.32 xmax: 572661.4 ymax: 381202.3
Projected CRS: NAD_1983_HARN_StatePlane_Hawaii_1_FIPS_5101


In [18]:
grid_dt <- grid_dt[trees > 0.5 & !is.na(agbd_m) &!is.na(agbd_sd)& !is.na(aet)]

In [19]:
grid_dt$agbd_n <- grid_dt$agbd_n %>% as.numeric
grid_dt$agbd_sd <- grid_dt$agbd_sd %>% as.numeric
grid_dt$agbd_m <- grid_dt$agbd_m %>% as.numeric

In [39]:
fia_dt$SOC_Mg_Ha <- fia_dt$SOC_Mg_Ha %>% as.numeric

Warning message in fia_dt$SOC_Mg_Ha %>% as.numeric:
“NAs introduced by coercion”


In [40]:
grid_int_dt  <- fia_dt$geom %>% st_transform(4135) %>% st_buffer(10000) %>% st_transform(4326) %>% st_intersects(grid_dt$geom)

In [41]:
rm_vec <- (grid_int_dt %>% lapply(length) %>% unlist) > 0
grid_int_dt <- grid_int_dt[rm_vec]
fia_dt <- fia_dt[rm_vec,]

In [42]:
names(grid_dt)

[1] "id"                 "left"               "top"               
 [4] "right"              "bottom"             "water"             
 [7] "trees"              "grass"              "flooded_vegetation"
[10] "crops"              "shrub_and_scrub"    "built"             
[13] "bare"               "snow_and_ice"       "elevation"         
[16] "landform"           "SRTM_mTPI"          "aet"               
[19] "def"                "pdsi"               "pet"               
[22] "pr"                 "ro"                 "soil"              
[25] "srad"               "swe"                "tmmn"              
[28] "tmmx"               "vap"                "vpd"               
[31] "vs"                 "agbd_m"             "agbd_sd"           
[34] "agbd_n"             "majorcomposition"   "age_years"         
[37] "Age_Class"          "geom"               "age_class"

In [43]:
grid_dt$age_years[grid_dt$age_years %>% is.na ] <- -2
grid_dt$age_class[grid_dt$age_class %>% is.na ] <- 'Unknown'

Warning message in `[<-.factor`(`*tmp*`, grid_dt$age_class %>% is.na, value = "Unknown"):
“invalid factor level, NA generated”


In [44]:
grid_dt %>% names

[1] "id"                 "left"               "top"               
 [4] "right"              "bottom"             "water"             
 [7] "trees"              "grass"              "flooded_vegetation"
[10] "crops"              "shrub_and_scrub"    "built"             
[13] "bare"               "snow_and_ice"       "elevation"         
[16] "landform"           "SRTM_mTPI"          "aet"               
[19] "def"                "pdsi"               "pet"               
[22] "pr"                 "ro"                 "soil"              
[25] "srad"               "swe"                "tmmn"              
[28] "tmmx"               "vap"                "vpd"               
[31] "vs"                 "agbd_m"             "agbd_sd"           
[34] "agbd_n"             "majorcomposition"   "age_years"         
[37] "Age_Class"          "geom"               "age_class"

In [45]:
grid_dt$age_class %>%  is.na %>% table

.
 TRUE 
67102 

In [46]:
grid_dt$max <- NULL
grid_dt$landform <-  grid_dt$landform %>% factor
grid_dt$majorcomposition <- grid_dt$majorcomposition %>% factor
grid_dt$age_class <- grid_dt$age_class %>% factor

Warning message in set(x, j = name, value = value):
“Column 'max' does not exist to remove”


In [55]:
fia_dt$DEPTH_UPPER <- fia_dt$LAYER_TYPE %>% gsub('MINERAL_SOIL_','',.) %>% gsub('_CM','',.) %>% strsplit('_') %>% lapply(function(x){x[1]}) %>% unlist %>% as.numeric
fia_dt$DEPTH_LOWER <- fia_dt$LAYER_TYPE %>% gsub('MINERAL_SOIL_','',.) %>% gsub('_CM','',.) %>% strsplit('_') %>% lapply(function(x){x[2]}) %>% unlist %>% as.numeric

In [60]:
  rf_dt <- data.table(fia_dt[,c('PLOT','SOC_Mg_Ha','DEPTH_UPPER','DEPTH_LOWER','county_name')], grid_dt[grid_int_dt  %>%   lapply(function(x){ x[sample(1:length(x),1,replace=T)] %>% unlist}) %>% unlist,])

  x_var <- c('DEPTH_UPPER','DEPTH_LOWER',(rf_dt %>% names)[-c(1,2,3,4,5,6,7,8,9,10,ncol(rf_dt)-1)])
  print(x_var)
  # rf_form <- as.formula(paste0('Stock_SOC_Mg_Ha~',paste(x_var,collapse = '+')))

 [1] "DEPTH_UPPER"        "DEPTH_LOWER"        "water"             
 [4] "trees"              "grass"              "flooded_vegetation"
 [7] "crops"              "shrub_and_scrub"    "built"             
[10] "bare"               "snow_and_ice"       "elevation"         
[13] "landform"           "SRTM_mTPI"          "aet"               
[16] "def"                "pdsi"               "pet"               
[19] "pr"                 "ro"                 "soil"              
[22] "srad"               "swe"                "tmmn"              
[25] "tmmx"               "vap"                "vpd"               
[28] "vs"                 "agbd_m"             "agbd_sd"           
[31] "agbd_n"             "majorcomposition"   "age_years"         
[34] "Age_Class"          "age_class"         


In [ ]:
# for( i in 2:11){
cl <-makeCluster(8)
sim_dt <-
1:100 %>% parLapply(cl,.,function(m,fia_dt,grid_dt,grid_int_dt){
  library(sf)
  library(magrittr)
  library(data.table)
  library(raster)
  library(exactextractr)
  library(terra)
  library(randomForest)


  rf_dt <- data.table(fia_dt[,c('PLOT','SOC_Mg_Ha','DEPTH_UPPER','DEPTH_LOWER','county_name')], grid_dt[grid_int_dt  %>%   lapply(function(x){ x[sample(1:length(x),1,replace=T)] %>% unlist}) %>% unlist,])

  x_var <- c('DEPTH_UPPER','DEPTH_LOWER',(rf_dt %>% names)[-c(1,2,3,4,5,6,7,8,9,10,ncol(rf_dt)-1)])
  # print(x_var)
  rf_form <- as.formula(paste0('SOC_Mg_Ha~',paste(x_var,collapse = '+')))

  # rf_dt <-  rf_dt[complete.cases(rf_dt[,-c(ncol(rf_dt)),with=F]),]
  rf_dt <-  rf_dt[!is.na(Stock_SOC_Mg_Ha),]

  # print(nrows(rf_dt))
  rf_terra <- randomForest(rf_form,rf_dt,verbose=T)

  rf_imp_dt <- rf_terra$importance %>% t %>% data.table

  names(rf_imp_dt) <-paste0('var_imp_',names(rf_imp_dt))

  out_dt <- data.table(rf_dt,data.table(predicted = rf_terra$predicted),rf_imp_dt[rep(1,nrow(rf_dt)),])
  out_dt$r2 <- rf_terra$rsq[500]
  return(out_dt[county_name %in% 'Hawaii'])
  },fia_dt,grid_dt,grid_int_dt) %>%
  rbindlist
stopCluster(cl)

# sim_dt %>% st_write(paste0("/content/sim_results_Hawaii_10000m_Run",i,".gpkg"), append=FALSE)

# drive_upload(paste0("/content/sim_results_Hawaii_10000m_Run",i,".gpkg"), path = as_id("1uVZCzCb5gRE8wfZckspRuKz1TQd5LpeM"))
# }

In [ ]:
sim_dt %>% head

In [ ]:
# Write to GeoPackage
sim_dt %>% st_write("/content/sim_results_Hawaii_10000m_Run1.gpkg", append=FALSE)

In [ ]:
# https://drive.google.com/drive/folders/1uVZCzCb5gRE8wfZckspRuKz1TQd5LpeM?usp=drive_link
drive_upload("/content/sim_results_Hawaii_10000m_Run1.gpkg", path = as_id("1uVZCzCb5gRE8wfZckspRuKz1TQd5LpeM"))

In [ ]:
# sim_dt

In [ ]:
sim_dt %>%
  ggplot(aes(y=Stock_SOC_Mg_Ha,x=r2))+
    geom_bin2d()
    # geom_smooth()